In [2]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
import pandas as pd

f = "/data/jlai/iris-hep/output100/estimatedparams.root"
file = uproot.open(f)
t = file['estimatedparams']
t.keys()

['event_nr',
 'volumeId',
 'layerId',
 'surfaceId',
 'loc0',
 'loc1',
 'phi',
 'theta',
 'qop',
 'time',
 'err_loc0',
 'err_loc1',
 'err_phi',
 'err_theta',
 'err_qop',
 'err_time',
 'charge',
 'p',
 'pt',
 'eta',
 'truthMatched',
 'particleId',
 'nMajorityHits',
 'particleId',
 't_loc0',
 't_loc1',
 't_phi',
 't_theta',
 't_qop',
 't_time',
 't_charge',
 't_p',
 't_pt',
 't_eta',
 'res_loc0',
 'res_loc1',
 'res_phi',
 'res_theta',
 'res_qop',
 'res_time',
 'pull_loc0',
 'pull_loc1',
 'pull_phi',
 'pull_theta',
 'pull_qop',
 'pull_time']

In [6]:
# study_estimatedparams.py
# Requirements: uproot, numpy, pandas, matplotlib

import os, sys, math
import numpy as np
import pandas as pd
import uproot
import matplotlib.pyplot as plt

# ----------------------------
# User inputs
# ----------------------------
ROOT_PATH = "/data/jlai/iris-hep/OutputPT/output_pt_10/estimatedparams.root"   # <-- set this
OUTPUT_DIR = "./estparams_study"

# ----------------------------
# Helper
# ----------------------------
PARAMS = [
    # (name, err_col, truth_col, reco_col)
    ("loc0",  "err_loc0",  "t_loc0",  "loc0"),
    ("loc1",  "err_loc1",  "t_loc1",  "loc1"),
    ("phi",   "err_phi",   "t_phi",   "phi"),
    ("theta", "err_theta", "t_theta", "theta"),
    ("qop",   "err_qop",   "t_qop",   "qop"),
    ("time",  "err_time",  "t_time",  "time"),
]

STORED_RES_COLS  = [f"res_{p[0]}"  for p in PARAMS]
STORED_PULL_COLS = [f"pull_{p[0]}" for p in PARAMS]

META_COLS = [
    "event_nr", "volumeId", "layerId", "surfaceId",
    "eta", "pt", "p", "qop", "theta", "phi",
    "truthMatched", "particleId", "nMajorityHits"
]

def find_tree(file):
    # Heuristic: take the first TTree in the file if you don’t know the exact name
    with uproot.open(file) as f:
        keys = [k for k in f.keys()]
        for k in keys:
            try:
                obj = f[k]
                if hasattr(obj, "num_entries"):
                    return k
            except Exception:
                pass
    raise RuntimeError("No TTree found in file.")

def safe_cols(all_cols, wanted):
    return [c for c in wanted if c in all_cols]

def finite_mask(x):
    return np.isfinite(x)

# ----------------------------
# Load
# ----------------------------
os.makedirs(OUTPUT_DIR, exist_ok=True)

tree_name = find_tree(ROOT_PATH)
print(f"Using tree: {tree_name}")

with uproot.open(ROOT_PATH) as f:
    tree = f[tree_name]
    all_branches = tree.keys()

# Decide which columns to read
needed_cols = set()
for name, err, tcol, rcol in PARAMS:
    needed_cols.update([err, tcol, rcol, f"res_{name}", f"pull_{name}"])
needed_cols.update(META_COLS)

cols_to_read = safe_cols(all_branches, list(needed_cols))
missing = sorted(list(set(needed_cols) - set(cols_to_read)))
if missing:
    print("⚠️ Missing columns (will skip where needed):", missing)

df = tree.arrays(cols_to_read, library="pd")

# ----------------------------
# Recompute residuals/pulls and compare with stored
# Convention check:
#   Some pipelines define residual as (reco - truth), others as (truth - reco).
#   We try both and see which matches the stored res_* best.
# ----------------------------
results = []
summary_rows = []

for name, err, tcol, rcol in PARAMS:
    res_col  = f"res_{name}"
    pull_col = f"pull_{name}"

    # Skip if either truth or reco missing
    if tcol not in df.columns or rcol not in df.columns:
        print(f"Skipping {name}: missing {tcol} or {rcol}")
        continue

    # Compute candidate residuals
    res_reco_minus_truth  = df[rcol] - df[tcol]
    res_truth_minus_reco  = df[tcol] - df[rcol]

    # If stored residual exists, measure which convention matches better
    if res_col in df.columns:
        d1 = np.nanmean(np.abs(df[res_col] - res_reco_minus_truth))
        d2 = np.nanmean(np.abs(df[res_col] - res_truth_minus_reco))
        use_rmt = d1 <= d2  # True if (reco - truth) fits better
    else:
        # If no stored residuals, default to reco - truth
        use_rmt = True

    # Choose convention
    res = res_reco_minus_truth if use_rmt else res_truth_minus_reco
    res_conv = "(reco - truth)" if use_rmt else "(truth - reco)"

    # Pulls
    if err in df.columns:
        # Guard against zero/NaN errors
        errsafe = df[err].replace(0, np.nan)
        pull = res / errsafe
    else:
        pull = pd.Series([np.nan]*len(df), index=df.index)

    # Compare to stored res/pull if present
    res_match = pull_match = np.nan
    if res_col in df.columns:
        msk = finite_mask(df[res_col].to_numpy()) & finite_mask(res.to_numpy())
        if msk.any():
            res_match = np.nanmean(np.abs(df.loc[msk, res_col] - res[msk]))
    if pull_col in df.columns:
        msk = finite_mask(df[pull_col].to_numpy()) & finite_mask(pull.to_numpy())
        if msk.any():
            pull_match = np.nanmean(np.abs(df.loc[msk, pull_col] - pull[msk]))

    # Attach computed columns (non-destructive names)
    df[f"res_{name}_recomputed"]  = res
    df[f"pull_{name}_recomputed"] = pull

    # Global stats
    r = res.to_numpy()
    pm = pull.to_numpy()

    res_mu  = np.nanmean(r)
    res_sig = np.nanstd(r)
    pull_mu = np.nanmean(pm)
    pull_sd = np.nanstd(pm)

    summary_rows.append({
        "param": name,
        "res_convention": res_conv,
        "mean(res)": res_mu,
        "std(res)": res_sig,
        "mean(pull)": pull_mu,
        "std(pull)": pull_sd,
        "mean_abs_diff_vs_stored_res": res_match,
        "mean_abs_diff_vs_stored_pull": pull_match,
        "have_err": err in df.columns,
        "have_stored_res": res_col in df.columns,
        "have_stored_pull": pull_col in df.columns,
    })

# Save global summary
summary_df = pd.DataFrame(summary_rows)
summary_csv = os.path.join(OUTPUT_DIR, "summary_global.csv")
summary_df.to_csv(summary_csv, index=False)
print(f"Saved {summary_csv}")
print(summary_df)

# ----------------------------
# Grouped summaries (by geometry IDs and by |eta| bins)
# ----------------------------
def grouped_stats(df, group_cols, name):
    rows = []
    for param, _, _, _ in PARAMS:
        res_col  = f"res_{param}_recomputed"
        pull_col = f"pull_{param}_recomputed"
        have = (res_col in df.columns)

        if not have:
            continue

        grp = df.groupby(group_cols)
        tmp = grp[res_col].agg(['count', 'mean', 'std']).reset_index()
        tmp.columns = group_cols + [f"{param}_count", f"{param}_res_mean", f"{param}_res_std"]

        # Pulls (only if exists)
        if pull_col in df.columns:
            tmp2 = grp[pull_col].agg(['mean','std']).reset_index()
            tmp2.columns = group_cols + [f"{param}_pull_mean", f"{param}_pull_std"]
            tmp = pd.merge(tmp, tmp2, on=group_cols, how='left')

        rows.append(tmp)

    if not rows:
        return None

    out = rows[0]
    for k in rows[1:]:
        out = pd.merge(out, k, on=group_cols, how='outer')

    out_csv = os.path.join(OUTPUT_DIR, f"summary_by_{name}.csv")
    out.to_csv(out_csv, index=False)
    print(f"Saved {out_csv}")
    return out

# by volume/layer/surface
grouped_stats(df, ["volumeId","layerId","surfaceId"], "volume_layer_surface")

# by |eta| bins
if "eta" in df.columns:
    eta_bins = np.array([-math.inf, -2.5, -1.5, -0.5, 0.5, 1.5, 2.5, math.inf])
    df["_eta_bin"] = pd.cut(df["eta"], bins=eta_bins)
    grouped_stats(df, ["_eta_bin"], "eta_bins")

# ----------------------------
# Quick plots: residuals & pulls
# ----------------------------
def plot_hist(series, title, fname, bins=100):
    arr = series.replace([np.inf, -np.inf], np.nan).dropna().to_numpy()
    if arr.size == 0:
        print(f"Skip plot {title}: empty")
        return
    plt.figure(figsize=(7,5))
    plt.hist(arr, bins=bins)
    plt.title(title)
    plt.xlabel("value")
    plt.ylabel("entries")
    plt.tight_layout()
    plt.savefig(fname, dpi=120)
    plt.close()
    print(f"Saved {fname}")

for name, _, _, _ in PARAMS:
    rrc = f"res_{name}_recomputed"
    prc = f"pull_{name}_recomputed"
    if rrc in df.columns:
        plot_hist(df[rrc], f"Residual: {name}", os.path.join(OUTPUT_DIR, f"hist_res_{name}.png"))
    if prc in df.columns:
        plot_hist(df[prc], f"Pull: {name}", os.path.join(OUTPUT_DIR, f"hist_pull_{name}.png"))

# ----------------------------
# Sanity printout of convention agreement
# ----------------------------
print("\n=== Convention check vs stored residuals/pulls ===")
for _, row in summary_df.iterrows():
    print(
        f"{row['param']:>6s} | conv: {row['res_convention']:<16s} "
        f"| mean|Δres| vs stored: {row['mean_abs_diff_vs_stored_res']!s:<12} "
        f"| mean|Δpull| vs stored: {row['mean_abs_diff_vs_stored_pull']!s:<12}"
    )

print("\nDone.")


Using tree: estimatedparams;1
Saved ./estparams_study/summary_global.csv
   param  res_convention     mean(res)   std(res)  mean(pull)  std(pull)  \
0   loc0  (reco - truth) -5.889615e-05   0.014867   -0.000059   0.014867   
1   loc1  (reco - truth) -1.313404e-05   0.015221   -0.000013   0.015221   
2    phi  (reco - truth)  1.481739e-04   0.041222    0.008490   2.361874   
3  theta  (reco - truth)  2.810775e-07   0.000324    0.000016   0.018557   
4    qop  (reco - truth)  6.298364e-05   0.023880    0.000625   0.237312   
5   time  (reco - truth)  1.109882e-01  24.832134    0.000370   0.082831   

   mean_abs_diff_vs_stored_res  mean_abs_diff_vs_stored_pull  have_err  \
0                      0.00000                      0.000000      True   
1                      0.00000                      0.000000      True   
2                      0.00027                      0.015496      True   
3                      0.00000                      0.000000      True   
4                      0

/tmp/ipykernel_4052502/784965816.py:183: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = df.groupby(group_cols)
/tmp/ipykernel_4052502/784965816.py:183: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = df.groupby(group_cols)
/tmp/ipykernel_4052502/784965816.py:183: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = df.groupby(group_cols)
/tmp/ipykernel_4052502/784965816.py:183: FutureWarning: The default of obser

Saved ./estparams_study/summary_by_eta_bins.csv
Saved ./estparams_study/hist_res_loc0.png
Saved ./estparams_study/hist_pull_loc0.png
Saved ./estparams_study/hist_res_loc1.png
Saved ./estparams_study/hist_pull_loc1.png
Saved ./estparams_study/hist_res_phi.png
Saved ./estparams_study/hist_pull_phi.png
Saved ./estparams_study/hist_res_theta.png
Saved ./estparams_study/hist_pull_theta.png
Saved ./estparams_study/hist_res_qop.png
Saved ./estparams_study/hist_pull_qop.png
Saved ./estparams_study/hist_res_time.png
Saved ./estparams_study/hist_pull_time.png

=== Convention check vs stored residuals/pulls ===
  loc0 | conv: (reco - truth)   | mean|Δres| vs stored: 0.0          | mean|Δpull| vs stored: 0.0         
  loc1 | conv: (reco - truth)   | mean|Δres| vs stored: 0.0          | mean|Δpull| vs stored: 0.0         
   phi | conv: (reco - truth)   | mean|Δres| vs stored: 0.0002704494399949908 | mean|Δpull| vs stored: 0.015495611354708672
 theta | conv: (reco - truth)   | mean|Δres| vs stored